In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install optuna tqdm ipywidgets pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 92.2 MB/s eta 0:00:00


In [ ]:
# =========================================================
# Import
# =========================================================
import os, json, warnings, math, random, pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    average_precision_score, roc_auc_score, f1_score, accuracy_score,
    precision_recall_curve
)
import optuna
warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Device = {DEVICE}")

from pytorch_tabnet.tab_model import TabNetClassifier

# =========================================================
# 파일 경로
# =========================================================
FILE_PATHS = {
    "Amazon":  '/content/drive/MyDrive/1014/data/new_amazon.csv',
    "Coursera":'/content/drive/MyDrive/1014/data/new_coursera.csv',
    "Audible": '/content/drive/MyDrive/1014/data/new_audible.csv',
    "Hotel":   '/content/drive/MyDrive/1014/data/new_hotel.csv'
}

# =========================================================
# S1 피처 (그대로 사용, 없는 컬럼은 자동 제외)
# =========================================================
S1_FEATURES = {
    "Amazon":  ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Price','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Is_Photo','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Coursera":['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Num_of_Reviews','Num_of_Enrolled','Num_of_top_instructor_courses','Num_of_top_instructor_learners','Text_Length','Valence','Arousal','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Audible": ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Hotel":   ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth','Is_Photo','Hotel_Grade','Employee_Friendliness_Score','Facility_Score','Cleanliness_Score','Comfort_Score','Value_For_Money_Score','Location_Score']
}

# =========================================================
# Config
# =========================================================
TARGET_COLUMN     = 'binary_helpfulness'
TEST_SPLIT_RATIO  = 0.2
RANDOM_STATE      = 42
N_TRIALS          = 50

def set_seed(seed=RANDOM_STATE):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(RANDOM_STATE)

# =========================================================
# Utils
# =========================================================
def _make_numeric_df(df: pd.DataFrame) -> pd.DataFrame:
    num = df.apply(pd.to_numeric, errors='coerce')
    med = num.median()
    return num.fillna(med)

def find_best_threshold(y_true, prob):
    precision, recall, thresholds = precision_recall_curve(y_true, prob)
    thresholds = np.concatenate([thresholds, [1.0]])
    f1s = (2 * precision * recall) / np.clip(precision + recall, 1e-9, None)
    idx = int(np.nanargmax(f1s))
    return float(thresholds[idx]), float(f1s[idx])

# =========================================================
# TabNet 학습 함수 (한 번의 학습)
# =========================================================
def fit_tabnet_once(X_tr, y_tr, X_va, y_va, params, max_epochs=200, patience=30, batch_size=1024):
    """
    TabNet 한 번 학습(early stopping 내장). 학습 후 valid PR_AUC 계산해서 반환.
    """
    clf = TabNetClassifier(
        n_d=params.get("n_d", 32),
        n_a=params.get("n_a", 32),
        n_steps=params.get("n_steps", 3),
        gamma=params.get("gamma", 1.5),
        n_independent=params.get("n_independent", 1),
        n_shared=params.get("n_shared", 1),
        momentum=params.get("momentum", 0.3),
        lambda_sparse=params.get("lambda_sparse", 1e-4),
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=params.get("lr", 2e-3), weight_decay=params.get("weight_decay", 1e-5)),
        scheduler_params={"step_size": 50, "gamma": 0.9},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type=params.get("mask_type", "sparsemax"),  # or "entmax"
        seed=RANDOM_STATE,
        device_name=("cuda" if DEVICE=="cuda" else "cpu"),
        verbose=0
    )

    # TabNet은 내부 metric으로 pr_auc가 없어서, early stopping은 'auc' 로 하고
    # 튜닝/평가는 외부에서 PR_AUC로 측정
    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_name=['valid'],
        eval_metric=['auc'],
        max_epochs=max_epochs,
        patience=patience,
        batch_size=batch_size,
        virtual_batch_size=min(256, batch_size),
        num_workers=0,
        drop_last=False
    )

    # validation PR_AUC 계산
    va_prob = clf.predict_proba(X_va)[:, 1]
    pr_auc = average_precision_score(y_va, va_prob)
    return clf, float(pr_auc)

# =========================================================
# 메인 파이프라인 (플랫폼 단위)
# =========================================================
def run_s1_pipeline(platform, csv_path, features):
    print("\n" + "="*60)
    print(f"▶ Platform: {platform}")
    print("="*60)

    # 1) 데이터 로드 & 전처리
    df = pd.read_csv(csv_path)
    assert TARGET_COLUMN in df.columns, f"{TARGET_COLUMN} 없음"
    labels = df[TARGET_COLUMN].astype(int).values

    exists = [c for c in features if c in df.columns]
    if len(exists)==0:
        raise ValueError("사용 가능한 S1 피처가 없습니다.")
    X_all = _make_numeric_df(df[exists]).to_numpy()

    # Stratified split
    idx = np.arange(len(df))
    tr_idx, te_idx = train_test_split(
        idx, test_size=TEST_SPLIT_RATIO, random_state=RANDOM_STATE, stratify=labels
    )
    X_train_raw, X_test_raw = X_all[tr_idx], X_all[te_idx]
    y_train, y_test = labels[tr_idx], labels[te_idx]

    # 스케일링(Optional) — TabNet은 스케일 필수는 아니지만, 안정적 수렴 위해 표준화 권장
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_raw)
    X_test  = scaler.transform(X_test_raw)
    print(f"✅ Train={len(y_train)}, Test={len(y_test)} | d={X_train.shape[1]}")

    # 2) Optuna (PR_AUC 최대화) — 3-fold CV
    def objective(trial):
        params = {
            "n_d": trial.suggest_categorical("n_d", [16, 24, 32, 48, 64]),
            "n_a": trial.suggest_categorical("n_a", [16, 24, 32, 48, 64]),
            "n_steps": trial.suggest_int("n_steps", 3, 7),
            "gamma": trial.suggest_float("gamma", 1.0, 2.5),
            "n_independent": trial.suggest_int("n_independent", 1, 3),
            "n_shared": trial.suggest_int("n_shared", 1, 3),
            "momentum": trial.suggest_float("momentum", 0.01, 0.4),
            "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True),
            "lr": trial.suggest_float("lr", 5e-4, 5e-3, log=True),
            "weight_decay": trial.suggest_float("weight_decay", 1e-7, 1e-3, log=True),
            "mask_type": trial.suggest_categorical("mask_type", ["sparsemax", "entmax"]),
            "batch_size": trial.suggest_categorical("batch_size", [512, 1024, 2048]),
            "max_epochs": 200,
            "patience": 30
        }

        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
        pr_aucs = []
        for tr, va in skf.split(X_train, y_train):
            X_tr, X_va = X_train[tr], X_train[va]
            y_tr, y_va = y_train[tr], y_train[va]

            model, val_pr = fit_tabnet_once(
                X_tr, y_tr, X_va, y_va,
                params,
                max_epochs=params["max_epochs"],
                patience=params["patience"],
                batch_size=params["batch_size"]
            )
            pr_aucs.append(val_pr)

        return float(np.mean(pr_aucs))

    study = optuna.create_study(direction="maximize")
    with tqdm(total=N_TRIALS, desc=f"Optuna Tuning [{platform}] (TabNet)", unit="trial") as pbar:
        def cb(study, trial):
            pbar.update(1)
        study.optimize(objective, n_trials=N_TRIALS, callbacks=[cb])

    best_params = study.best_params
    print("🧪 Best Params (TabNet):", best_params)

    # 3) 최종 학습 (train 내부 10%를 val로 사용해 early stopping)
    sub_tr, val = train_test_split(
        np.arange(len(y_train)), test_size=0.1, random_state=RANDOM_STATE, stratify=y_train
    )
    X_sub, y_sub = X_train[sub_tr], y_train[sub_tr]
    X_val, y_val = X_train[val],   y_train[val]

    final_model, _ = fit_tabnet_once(
        X_sub, y_sub, X_val, y_val,
        {**best_params},
        max_epochs=best_params.get("max_epochs", 200),
        patience=best_params.get("patience", 30),
        batch_size=best_params.get("batch_size", 1024)
    )

    # 4) 평가 + threshold
    test_prob = final_model.predict_proba(X_test)[:, 1]
    train_prob = final_model.predict_proba(X_train)[:, 1]
    best_th, _ = find_best_threshold(y_train, train_prob)
    test_pred = (test_prob >= best_th).astype(int)

    metrics = {
        "Accuracy": float(accuracy_score(y_test, test_pred)),
        "PR_AUC":   float(average_precision_score(y_test, test_prob)),
        "ROC_AUC":  float(roc_auc_score(y_test, test_prob)),
        "F1_score": float(f1_score(y_test, test_pred)),
        "Best_Threshold": float(best_th)
    }
    print("=== Test Metrics (TabNet) ===", metrics)

    # 5) 저장
    save_dir = f"/content/drive/MyDrive/1014/result_tabnet/{platform}"
    os.makedirs(save_dir, exist_ok=True)

    # 확률/예측
    pd.DataFrame({
        "index": te_idx,
        "s1_pred_proba": test_prob,
        "y_true": y_test,
        "y_pred_at_best_th": test_pred
    }).to_csv(f"{save_dir}/s1_pred_proba.csv", index=False)

    # 메트릭 + 설정
    with open(f"{save_dir}/results.json", "w") as f:
        json.dump({
            **metrics,
            "features_used": exists,
            "random_state": RANDOM_STATE,
            "best_params": best_params
        }, f, indent=2)

    # 모델 저장 (TabNet은 .zip으로 export_model 제공)
    model_path = f"{save_dir}/tabnet_model.zip"
    final_model.save_model(model_path)
    # 스케일러 저장
    with open(f"{save_dir}/scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)

    print(f"📁 Results saved in {save_dir}")
    print(f"🧠 Model: {model_path}  |  Scaler: {save_dir}/scaler.pkl")

# =========================================================
# 전체 실행 (플랫폼 루프도 notebook tqdm)
# =========================================================
for platform, path in tqdm(FILE_PATHS.items(), desc="전체 플랫폼 진행 (TabNet)", unit="platform"):
    run_s1_pipeline(platform, path, S1_FEATURES[platform])

ModuleNotFoundError: No module named 'optuna'

In [ ]:
!pip -q install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 30.1 MB/s eta 0:00:00


# fold 안한 버전

In [ ]:
# =========================================================
# Fast TabNet (no K-Fold, fewer epochs)
# =========================================================
import os, json, warnings, random, pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    average_precision_score, roc_auc_score, f1_score, accuracy_score,
    precision_recall_curve
)
import optuna
warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Device = {DEVICE}")

from pytorch_tabnet.tab_model import TabNetClassifier

# -----------------------------
# 파일 경로
# -----------------------------
FILE_PATHS = {
    "Amazon":  '/content/drive/MyDrive/1014/data/new_amazon.csv',
    "Coursera":'/content/drive/MyDrive/1014/data/new_coursera.csv',
    "Audible": '/content/drive/MyDrive/1014/data/new_audible.csv',
    "Hotel":   '/content/drive/MyDrive/1014/data/new_hotel.csv'
}

# =========================================================
# S1 피처 (그대로 사용, 없는 컬럼은 자동 제외)
# =========================================================
S1_FEATURES = {
    "Amazon":  ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Price','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Is_Photo','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Coursera":['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Num_of_Reviews','Num_of_Enrolled','Num_of_top_instructor_courses','Num_of_top_instructor_learners','Text_Length','Valence','Arousal','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Audible": ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth'],
    "Hotel":   ['Average_Rating','Rating','Deviation_Of_Star_Ratings','Time_Lapsed','Text_Length','Valence','Arousal','Title_Length','Num_of_Ratings','Flesch_Reading_Ease','FOG_Index','Sentiment_Score','new_depth','new_breadth','Is_Photo','Hotel_Grade','Employee_Friendliness_Score','Facility_Score','Cleanliness_Score','Comfort_Score','Value_For_Money_Score','Location_Score']
}


# -----------------------------
# Config (빠른 실행용)
# -----------------------------
TARGET_COLUMN     = 'binary_helpfulness'
TEST_SPLIT_RATIO  = 0.2
RANDOM_STATE      = 42
N_TRIALS          = 30            # 🔽 빠르게: 10회(원하면 5로 더 줄여도 됨)
MAX_EPOCHS_FAST   = 60            # 🔽 60 epoch
PATIENCE_FAST     = 10            # 🔽 patience 10
BATCH_CHOICES     = [512, 1024]   # 🔽 batch 후보 축소

def set_seed(seed=RANDOM_STATE):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(RANDOM_STATE)

def _make_numeric_df(df: pd.DataFrame) -> pd.DataFrame:
    num = df.apply(pd.to_numeric, errors='coerce')
    return num.fillna(num.median())

def find_best_threshold(y_true, prob):
    precision, recall, thresholds = precision_recall_curve(y_true, prob)
    thresholds = np.concatenate([thresholds, [1.0]])
    f1s = (2 * precision * recall) / np.clip(precision + recall, 1e-9, None)
    idx = int(np.nanargmax(f1s))
    return float(thresholds[idx]), float(f1s[idx])

def fit_tabnet_once(X_tr, y_tr, X_va, y_va, params,
                    max_epochs=MAX_EPOCHS_FAST, patience=PATIENCE_FAST, batch_size=1024):
    clf = TabNetClassifier(
        n_d=params.get("n_d", 32),
        n_a=params.get("n_a", 32),
        n_steps=params.get("n_steps", 3),
        gamma=params.get("gamma", 1.5),
        n_independent=params.get("n_independent", 1),
        n_shared=params.get("n_shared", 1),
        momentum=params.get("momentum", 0.3),
        lambda_sparse=params.get("lambda_sparse", 1e-4),
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=params.get("lr", 2e-3), weight_decay=params.get("weight_decay", 1e-5)),
        scheduler_params={"step_size": 30, "gamma": 0.9},  # 🔽 step 더 짧게
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type=params.get("mask_type", "sparsemax"),
        seed=RANDOM_STATE,
        device_name=("cuda" if DEVICE=="cuda" else "cpu"),
        verbose=0
    )
    clf.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_name=['valid'],
        eval_metric=['auc'],               # 내부는 AUC로 early stop
        max_epochs=max_epochs,
        patience=patience,
        batch_size=batch_size,
        virtual_batch_size=min(256, batch_size),
        num_workers=0,
        drop_last=False
    )
    va_prob = clf.predict_proba(X_va)[:, 1]
    pr_auc = average_precision_score(y_va, va_prob)  # 외부 평가는 PR-AUC
    return clf, float(pr_auc)

def run_s1_pipeline(platform, csv_path, features):
    print("\n" + "="*60)
    print(f"▶ Platform: {platform}")
    print("="*60)

    # 1) 데이터 & 전처리
    df = pd.read_csv(csv_path)
    assert TARGET_COLUMN in df.columns, f"{TARGET_COLUMN} 없음"
    y = df[TARGET_COLUMN].astype(int).values

    exists = [c for c in features if c in df.columns]
    X = _make_numeric_df(df[exists]).to_numpy()

    idx = np.arange(len(df))
    tr_idx, te_idx = train_test_split(
        idx, test_size=TEST_SPLIT_RATIO, random_state=RANDOM_STATE, stratify=y
    )
    X_train_raw, X_test_raw = X[tr_idx], X[te_idx]
    y_train, y_test = y[tr_idx], y[te_idx]

    # 표준화(수렴 안정)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_raw)
    X_test  = scaler.transform(X_test_raw)
    print(f"✅ Train={len(y_train)}, Test={len(y_test)} | d={X_train.shape[1]}")

    # 2) Optuna (빠른 모드: holdout만 사용)
    def objective(trial):
        params = {
            "n_d": trial.suggest_categorical("n_d", [24, 32, 48]),
            "n_a": trial.suggest_categorical("n_a", [24, 32, 48]),
            "n_steps": trial.suggest_int("n_steps", 3, 5),
            "gamma": trial.suggest_float("gamma", 1.0, 2.0),
            "n_independent": trial.suggest_int("n_independent", 1, 2),
            "n_shared": trial.suggest_int("n_shared", 1, 2),
            "momentum": trial.suggest_float("momentum", 0.05, 0.35),
            "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-6, 5e-4, log=True),
            "lr": trial.suggest_float("lr", 8e-4, 3e-3, log=True),
            "weight_decay": trial.suggest_float("weight_decay", 1e-7, 5e-4, log=True),
            "mask_type": trial.suggest_categorical("mask_type", ["sparsemax", "entmax"]),
            "batch_size": trial.suggest_categorical("batch_size", BATCH_CHOICES),
        }
        # ✅ 단일 holdout (fold X)
        tr_sub, va_sub = train_test_split(
            np.arange(len(y_train)), test_size=0.1, random_state=RANDOM_STATE, stratify=y_train
        )
        X_tr, y_tr = X_train[tr_sub], y_train[tr_sub]
        X_va, y_va = X_train[va_sub], y_train[va_sub]

        _, pr = fit_tabnet_once(
            X_tr, y_tr, X_va, y_va,
            params,
            max_epochs=MAX_EPOCHS_FAST,
            patience=PATIENCE_FAST,
            batch_size=params["batch_size"]
        )
        return pr

    study = optuna.create_study(direction="maximize")
    with tqdm(total=N_TRIALS, desc=f"Optuna Tuning [{platform}] (Fast TabNet)", unit="trial") as pbar:
        def _cb(study, trial): pbar.update(1)
        study.optimize(objective, n_trials=N_TRIALS, callbacks=[_cb])

    best_params = study.best_params
    print("🧪 Best Params (Fast TabNet):", best_params)

    # 3) 최종 학습 (train 90% / val 10%)
    sub_tr, val = train_test_split(
        np.arange(len(y_train)), test_size=0.1, random_state=RANDOM_STATE, stratify=y_train
    )
    X_sub, y_sub = X_train[sub_tr], y_train[sub_tr]
    X_val, y_val = X_train[val],   y_train[val]

    final_model, _ = fit_tabnet_once(
        X_sub, y_sub, X_val, y_val,
        {**best_params},
        max_epochs=MAX_EPOCHS_FAST,
        patience=PATIENCE_FAST,
        batch_size=best_params.get("batch_size", 1024)
    )

    # 4) 평가 + 임계값
    test_prob  = final_model.predict_proba(X_test)[:, 1]
    train_prob = final_model.predict_proba(X_train)[:, 1]
    best_th, _ = find_best_threshold(y_train, train_prob)
    test_pred  = (test_prob >= best_th).astype(int)

    metrics = {
        "Accuracy": float(accuracy_score(y_test, test_pred)),
        "PR_AUC":   float(average_precision_score(y_test, test_prob)),
        "ROC_AUC":  float(roc_auc_score(y_test, test_prob)),
        "F1_score": float(f1_score(y_test, test_pred)),
        "Best_Threshold": float(best_th)
    }
    print("=== Test Metrics (Fast TabNet) ===", metrics)

    # 5) 저장
    save_dir = f"/content/drive/MyDrive/1014/result_tabnet_fast/{platform}"
    os.makedirs(save_dir, exist_ok=True)

    pd.DataFrame({
        "index": te_idx,
        "s1_pred_proba": test_prob,
        "y_true": y_test,
        "y_pred_at_best_th": test_pred
    }).to_csv(f"{save_dir}/s1_pred_proba.csv", index=False)

    with open(f"{save_dir}/results.json", "w") as f:
        json.dump({
            **metrics,
            "features_used": exists,
            "random_state": RANDOM_STATE,
            "best_params": best_params,
            "fast_mode": {
                "n_trials": N_TRIALS,
                "max_epochs": MAX_EPOCHS_FAST,
                "patience": PATIENCE_FAST,
                "no_kfold": True
            }
        }, f, indent=2)

    # 모델 저장
    final_model.save_model(f"{save_dir}/tabnet_model.zip")

    # 스케일러 저장
    with open(f"{save_dir}/scaler.pkl", "wb") as f:
        pickle.dump(scaler, f)

# -----------------------------
# 전체 실행
# -----------------------------
for platform, path in tqdm(FILE_PATHS.items(), desc="전체 플랫폼 진행 (Fast TabNet)", unit="platform"):
    run_s1_pipeline(platform, path, S1_FEATURES[platform])

✅ Device = cuda


전체 플랫폼 진행 (Fast TabNet):   0%|          | 0/4 [00:00<?, ?platform/s]


▶ Platform: Amazon
✅ Train=71941, Test=17986 | d=16


Optuna Tuning [Amazon] (Fast TabNet):   0%|          | 0/30 [00:00<?, ?trial/s]


Early stopping occurred at epoch 43 with best_epoch = 33 and best_valid_auc = 0.77769
Stop training because you reached max_epochs = 60 with best_epoch = 57 and best_valid_auc = 0.79218
Stop training because you reached max_epochs = 60 with best_epoch = 59 and best_valid_auc = 0.79994
Stop training because you reached max_epochs = 60 with best_epoch = 57 and best_valid_auc = 0.79989
Stop training because you reached max_epochs = 60 with best_epoch = 56 and best_valid_auc = 0.81671

Early stopping occurred at epoch 34 with best_epoch = 24 and best_valid_auc = 0.78222

Early stopping occurred at epoch 47 with best_epoch = 37 and best_valid_auc = 0.78246

Early stopping occurred at epoch 28 with best_epoch = 18 and best_valid_auc = 0.77616

Early stopping occurred at epoch 46 with best_epoch = 36 and best_valid_auc = 0.78549

Early stopping occurred at epoch 51 with best_epoch = 41 and best_valid_auc = 0.78078
Stop training because you reached max_epochs = 60 with best_epoch = 58 and bes

...

Stop training because you reached max_epochs = 60 with best_epoch = 59 and best_valid_auc = 0.89324
Stop training because you reached max_epochs = 60 with best_epoch = 54 and best_valid_auc = 0.89663
Stop training because you reached max_epochs = 60 with best_epoch = 52 and best_valid_auc = 0.88261
Stop training because you reached max_epochs = 60 with best_epoch = 56 and best_valid_auc = 0.87501
Stop training because you reached max_epochs = 60 with best_epoch = 56 and best_valid_auc = 0.87488
Stop training because you reached max_epochs = 60 with best_epoch = 53 and best_valid_auc = 0.87071
Stop training because you reached max_epochs = 60 with best_epoch = 58 and best_valid_auc = 0.89719

Early stopping occurred at epoch 45 with best_epoch = 35 and best_valid_auc = 0.88545

Early stopping occurred at epoch 52 with best_epoch = 42 and best_valid_auc = 0.89135

Early stopping occurred at epoch 47 with best_epoch = 37 and best_valid_auc = 0.88973

Early stopping occurred at epoch 40 wi

Optuna Tuning [Audible] (Fast TabNet):   0%|          | 0/30 [00:00<?, ?trial/s]


Early stopping occurred at epoch 56 with best_epoch = 46 and best_valid_auc = 0.74933
Stop training because you reached max_epochs = 60 with best_epoch = 59 and best_valid_auc = 0.78594

Early stopping occurred at epoch 59 with best_epoch = 49 and best_valid_auc = 0.76266
Stop training because you reached max_epochs = 60 with best_epoch = 57 and best_valid_auc = 0.74905

Early stopping occurred at epoch 45 with best_epoch = 35 and best_valid_auc = 0.77281

Early stopping occurred at epoch 27 with best_epoch = 17 and best_valid_auc = 0.73147
Stop training because you reached max_epochs = 60 with best_epoch = 58 and best_valid_auc = 0.78674
Stop training because you reached max_epochs = 60 with best_epoch = 56 and best_valid_auc = 0.78321
Stop training because you reached max_epochs = 60 with best_epoch = 57 and best_valid_auc = 0.78065
Stop training because you reached max_epochs = 60 with best_epoch = 55 and best_valid_auc = 0.74228
Stop training because you reached max_epochs = 60 wi

Optuna Tuning [Hotel] (Fast TabNet):   0%|          | 0/30 [00:00<?, ?trial/s]


Early stopping occurred at epoch 40 with best_epoch = 30 and best_valid_auc = 0.67949

Early stopping occurred at epoch 50 with best_epoch = 40 and best_valid_auc = 0.6797

Early stopping occurred at epoch 35 with best_epoch = 25 and best_valid_auc = 0.66407
Stop training because you reached max_epochs = 60 with best_epoch = 59 and best_valid_auc = 0.69101

Early stopping occurred at epoch 17 with best_epoch = 7 and best_valid_auc = 0.6578
Stop training because you reached max_epochs = 60 with best_epoch = 55 and best_valid_auc = 0.69146

Early stopping occurred at epoch 40 with best_epoch = 30 and best_valid_auc = 0.66095

Early stopping occurred at epoch 48 with best_epoch = 38 and best_valid_auc = 0.67964
Stop training because you reached max_epochs = 60 with best_epoch = 53 and best_valid_auc = 0.69125

Early stopping occurred at epoch 54 with best_epoch = 44 and best_valid_auc = 0.69077

Early stopping occurred at epoch 41 with best_epoch = 31 and best_valid_auc = 0.69311

Early 